In [ ]:
#importing all necessary modules

import numpy as np

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

import math
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [ ]:
#1: selecting relevant features

master = [] #Initialize list to store the relevant data in

# List of files to be processed
files = ["19901999.csv", "20002012.csv", "20122014.csv", "20152016.csv", "2017onwards.csv"]

# Adds information for files in the first 4 sets. Last set has different indexing
for filename in files[:-1]:
  with open('Datasets/' + filename, 'r') as f:
    header = next(f)
    for row in f:
      lis = row.strip().split(',')
      lis[0] = lis[0][0:4] + lis[0][5:] #processing string with '-' into an integer value
      line = [float(lis[0]), lis[2], float(lis[6]), lis[7], float(lis[8]), float(lis[9])] #referencing relevent features and adding them
      line[3] = line[3].upper()
      master.append(line)

#Adds information for files in the last set
with open('Datasets/' + files[-1], 'r') as f:
  header = next(f)
  for row in f:
    lis = row.strip().split(',')
    lis[0] = lis[0][0:4] + lis[0][5:] #processing string with '-' into an integer value
    line = [float(lis[0]), lis[2], float(lis[6]), lis[7], float(lis[8]), float(lis[10])] #referencing relevent features and adding them
    line[3] = line[3].upper()
    master.append(line)

#Ignores data past 2020 
master = master[:810459]

#initialize values for better and more intuitive referencing of indexes with minimal confusion
month, flat_type, floor_area_sqm, flat_model, lease_commence_date, resale_price, unemployment_rate, real_gdp, distance_from_cbd, population_size = 0,1,2,3,4,5,6,7,8,9

#2: encoding categorical values for flat_model and flat_type
enc = OrdinalEncoder()
X = [['1 ROOM'], ['2 ROOM'], ['3 ROOM'], ['4 ROOM'], ['5 ROOM'], ['EXECUTIVE'], ['MULTI-GENERATION'], ['MULTI GENERATION']]
X_2 = [['SIMPLIFIED'], ['IMPROVED'], ['MODEL A'], ['APARTMENT'], ['NEW GENERATION'], ['STANDARD'], ['MAISONETTE'], ['MODEL A-MAISONETTE'], ['TERRACE'], ['IMPROVED-MAISONETTE'], ['PREMIUM APARTMENT'], ['MULTI GENERATION'], ['2-ROOM'], ['MODEL A2'], ['PREMIUM MAISONETTE'], ['ADJOINED FLAT'], ['TYPE S1'], ['DBSS'], ['PREMIUM APARTMENT LOFT'], ['TYPE S2'], ['3GEN']]

enc.fit(X)
result = enc.transform([[row[flat_type]] for row in master]) #flat type encoding
enc.fit(X_2)
result2 = enc.transform([[row[flat_model]] for row in master]) #flat model encoding

for row in range(len(master)):
  master[row][flat_type] = result[row][0]
  master[row][flat_model] = result2[row][0]



In [ ]:
#3: adding unemployment rate as a feature to the master dataset
import csv
#list of sublists seperated by quarters
months = [['01', '02', '03'], ['04', '05', '06'], ['07', '08', '09'], ['10', '11', '12']]

dic = {} #dic will store manually created string keys ('20241Q'...'19904Q') that will reference sublists consisting of months(from the months list) in that specific quarter.
#eg. {'20241Q':['202401.0', '202402.0', '202403.0']}
for i in range(1990, 2025):
  for j in range(1, 5):
    dic[str(i)+str(j)+'Q'] = []
    for m in months[j-1]:
      dic[str(i)+str(j)+'Q'] += [float(str(i)+m)]

finaldic = {}

with open('Datasets/UnemploymentRateEndOfPeriodQuarterlySeasonallyAdjusted.csv', 'r') as f:
  masterr = [] #this list will contain 2 rows: row 1 contains the quarterly label year values such as '20241Q'. row 2 contains the unemployment rate for that quarter such as '1.7'
  for row in f:
    row = row.strip().split(',')
    masterr.append(row[1:]) #removing the titles total unemployment rate, resident unemployment rate
  masterr = masterr[:2] #removing resident and citize unemployment rate
  masterr[1] = masterr[1][1:] #removing the title ' (SA)"'

#as the file only contains unemployment rates from 1992-2024,
#through research(https://stats.mom.gov.sg/iMAS_Tables/Times%20Series%20Table/mrsd_14_Historical_Unemployment_Rate_28Jan21.xlsx)
#we will be manually inserting unemployment rates as 1.7 for each quarter in 1990 and 1991.
for i in range(1, 5):
  masterr[0].append('1990'+str(i)+'Q')
  masterr[0].append('1991'+str(i)+'Q')
  masterr[1] += [1.7, 1.7]

#for each quarter label in the first row, and for each key in dic, if they equate to each other,
#we will insert into finaldic, (the values in the sublist that the key in dic refers to) as the key
#that references the (unemployment rate for that month)value.
for item in masterr[0]:
  for row in dic:
    if item == row:
      for i in dic[row]:
        finaldic[i] = float(masterr[1][masterr[0].index(item)])

#adding the correct unemployment rate to the correct month in the dataset based on the float key of the month in finaldic
for row in master:
  for item in finaldic:
    if row[month] == item:
      row += [finaldic[item]]


In [ ]:
#4: calculating remaining lease by performing the following: 99 - (current year - lease commencement date)
for row in master:
  result = 99-(int(str(row[month])[:4])-int(str(row[lease_commence_date])[:4]))
  row[lease_commence_date] = result

remaining_lease = lease_commence_date

In [ ]:
#5 inserting real gdp into the dataset
with open('Datasets/M015651.csv', 'r') as f:
  finalliz = f.readlines()
  year = (finalliz[10].strip().split(','))[1:] #removing irrelevant data and the header value to obtain the years
  r = year.index('1990 1Q ') #finding the index to cut off and obtain relevant years
  year = year[:r+1]
  gdp = (finalliz[11].strip().split(','))[1:] #removing irrelevant data and the header value to obtain the gdp
  gdp = gdp[:r+1]

for row in range(len(year)):
  year[row] = year[row][:4] + year[row][5:7] #removing spaces from the quearterly year labels

finalfinaldic = {}

#for each quarter label in the year list, and for each key in dic, if they equate to each other,
#we will insert into finalfinaldic, (the values in the sublist that the key in dic refers to) as the key
#that references the (gdp)value in the gdp list.
for item in year:
  for row in dic:
    if item == row:
      for i in dic[row]:
        finalfinaldic[i] = float(gdp[year.index(item)])

#adding the correct unemployment rate to the correct month in the dataset based on the float key of the month in finaldic
for row in master:
  for item in finalfinaldic:
    if row[month] == item:
      row += [finalfinaldic[item]]

In [ ]:
#6: inserting the distance to cbd

#Initializes working data array
data = []

for filename in files:
  with open("Datasets/" + filename,'r') as f:
    i = 0
    for row in f:
      if i != 0:
        data.append(row.strip('\n').split(','))
      i += 1

#Ignores data past 2020 
master = master[:810459]

def get_lat_lon(address):
    """
    Gets the latitude and longitude of the street using Nominatim API
    """
    #Initializes the geocoder and collects the location information
    geolocator = Nominatim(user_agent="Jupyter_AI_Project_HomeBros")
    location = geolocator.geocode(address, timeout=10)

    #Begins up to 5 attempts to find location, increasing delay time each attempt in case of timeout
    for attempt in range(5):
        delay = 1
        try:
            location = geolocator.geocode(address, timeout=10)

            #Checks if location found. If not, prints address and returns None.
            #When not found, address was examined to identify why. (During preprocessing. Resulted in the many if statements in the preprocessing function)
            if location:
                return location.latitude, location.longitude
            else:
                print(address)
                return None, None

        except GeocoderTimedOut:
            print(f"Timeout on attempt {attempt+1} for '{address}'... retrying in {delay}s.")
            time.sleep(delay)
            delay += 1


def haversine(lat1, lon1, lat2, lon2):
    """
    Calculates shortest distance to CBD in kilometres, accounting for the curve of the Earth
    """
    R = 6371  # Earth's radius in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return round(distance,3)

def preprocessing(data, streets, unique_streets):
  """
  Handles shortforms and missing streets
  """
  for row in data[1:]:
    new_strt = row[4]

    # Storage to retrieve indexes later
    if new_strt not in streets:
      streets.append(new_strt)

    # Processing
    if " NTH " in new_strt:
      new_strt = new_strt.replace(" NTH ", " NORTH ")
    if " NTH" in new_strt:
      new_strt = new_strt.replace(" NTH", " NORTH")
    if " STH " in new_strt:
      new_strt = new_strt.replace(" STH ", " SOUTH ")
    if " ST " in new_strt:
      new_strt = new_strt.replace(" ST ", " STREET ")
    if " RD " in new_strt:
      new_strt = new_strt.replace(" RD ", " ROAD ")
    if "BT " in new_strt:
      new_strt = new_strt.replace("BT ", "BUKIT ")
    if " BT " in new_strt:
      new_strt = new_strt.replace(" BT ", " BUKIT ")
    if " ST" == new_strt[-3:]:
      new_strt = new_strt[:-3] + " STREET"
    if " RD" == new_strt[-3:]:
      new_strt = new_strt[:-3] + " ROAD"
    if new_strt == "JLN MEMBINA BARAT":
      new_strt = "Central Green Condo" #Road no longer exists
    if "JLN " in new_strt:
      new_strt = new_strt.replace("JLN ", "JALAN ")
    if "LOR " in new_strt:
      new_strt = new_strt.replace("LOR ", "LORONG ")
    if " AVE " in new_strt:
      new_strt = new_strt.replace(" AVE ", " AVENUE ")
    if " AVE" == new_strt[-4:]:
      new_strt = new_strt[:-4] + " AVENUE"
    if " DR " in new_strt:
      new_strt = new_strt.replace(" DR ", " DRIVE ")
    if " DR" == new_strt[-3:]:
      new_strt = new_strt[:-3] + " DRIVE"
    if "C'WEALTH" in new_strt:
      new_strt = new_strt.replace("C'WEALTH", "COMMONWEALTH")
    if "TG " in new_strt:
      new_strt = new_strt.replace("TG ", "TANJONG ")
    if new_strt == "KG BAHRU HILL":
      new_strt = "SPOONER ROAD" #Road no longer exists
    elif "KG " in new_strt:
      new_strt = new_strt.replace("KG ", "KAMPONG ")
    if "UPP " in new_strt:
      new_strt = new_strt.replace("UPP ", "UPPER ")
    if "BUANGKOK SOUTH FARMWAY 1" == new_strt:
      new_strt = "BUANGKOK" #Road no longer exists

    #If street not already in unique_streets, add it
    if new_strt not in unique_streets:
      unique_streets.append(new_strt)
  return


#Selects unique streets for processing, and stores streets for returning to initial dataset
streets = []
unique_streets = []
preprocessing(data[1:], streets, unique_streets)
print("Streets split")

#Converts each item in streets to a list in the format [street_name, latitude, longitude]
i = 0
for row in unique_streets:
  address = row
  lat, lon = get_lat_lon(address)
  unique_streets[i] = [row, lat, lon]
  i += 1
print("Coordinates calculated")

# Initialized the values for CBD's latitude and longitude
CBD = [1.2812, 103.8503]
# Calculates distance to CBD of each location
for row in unique_streets:
  dist = haversine(CBD[0], CBD[1], row[1], row[2])
  row.append(dist)
print("Distance calculated")

# Adjusts main dataset to reflect dist to CBD instead of street name
for row in data:
  if streets.index(row[4]) > len(unique_streets):
    print("Error")
    print(row[4]) #Preprocessing checks
  else:
    record = unique_streets[streets.index(row[4])]
    row[4] = record[3]
print('Data attached to current dataset')

# Attached data to main working dataset
for row in range(len(data)):
  master[row].append(data[row][4])
print("Data attached to main dataset")

In [7]:
#7 Insert population into dataset
with open('Datasets/M810811.csv', 'r') as f:
  finalliz = f.readlines()
  year = (finalliz[10].strip().split(','))[1:] #removing irrelevant data and the header value to obtain the years
  population = (finalliz[11].strip().split(','))[1:] #removing irrelevant data and the header value to obtain the gdp

months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
finalpop = []
finalyear = []
for row in year:
  for m in months:
    hold = row.strip()+m
    if hold not in year:
      finalyear.append(hold)

for pop in population:
  for i in range(12):
    finalpop.append(pop)

#adding the correct unemployment rate to the correct month in the dataset based on the float key of the month in finaldic
for row in master:
  for item in finalyear:
    if row[month] == float(item):
      row += [float(finalpop[finalyear.index(item)])]


In [108]:
trainset = master[:596524] # 1990-2009
valset = master[596524:709050] # 2010-2014
testset = master[709050:810459] #2015-2019

In [113]:
#write to the storage dataset
def writefile(dataset, index):
    filesetname = ["19902009processedtesting", "20102014processedvalidation", "20152019processedtesting"]
    with open("Datasets/" + filesetname[index] + ".csv", "w") as f:
        for row in master:
            i = 0
            if len(row) == 10: #Checks for incomplete data. Print tests show data with wrong length is minimal enough to ignore altogether
                for each in row:
                    if type(each) != str:
                        each = str(each)
                    f.write(each)
                    i += 1
                    if i < 10:
                        f.write(",")
            f.write("\n")

writefile(trainset, 0)
writefile(valset, 1)
writefile(testset, 2)